## CIFAR-10 Image classification using Data Augmentation

#### Max accuracy get : 82 %

In [2]:
%matplotlib inline

import numpy as np
from sklearn.preprocessing import StandardScaler  

from matplotlib import pyplot as plt


from keras.utils import np_utils

from __future__ import absolute_import
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils


from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD

from keras.regularizers import l2, activity_l2



import theano.tensor.signal.conv 
from theano.tensor.signal import downsample

Using Theano backend.
/home/rishikesh/anaconda3/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [9]:
batch_size = 32
nb_classes = 10
nb_epoch = 200
data_augmentation = True

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3

In [10]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [11]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [12]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=(img_channels, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [8]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))

X_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/200
50000/50000 [==============================] - 164s - loss: 1.6982 - acc: 0.3701 - val_loss: 1.2920 - val_acc: 0.5253
Epoch 2/200
50000/50000 [==============================] - 169s - loss: 1.3551 - acc: 0.5100 - val_loss: 1.0992 - val_acc: 0.6136
Epoch 3/200
50000/50000 [==============================] - 169s - loss: 1.2194 - acc: 0.5613 - val_loss: 0.9819 - val_acc: 0.6466
Epoch 4/200
50000/50000 [==============================] - 169s - loss: 1.1222 - acc: 0.6006 - val_loss: 0.8905 - val_acc: 0.6834
Epoch 5/200
50000/50000 [==============================] - 165s - loss: 1.0561 - acc: 0.6266 - val_loss: 0.8472 - val_acc: 0.7019
Epoch 6/200
50000/50000 [==============================] - 175s - loss: 1.0108 - acc: 0.6436 - val_loss: 0.8883 - val_acc: 0.6886
Epoch 7/200
50000/50000 [==============================] - 172s - loss: 0.9876 - acc: 0.6506 - val_loss: 0.8214 

#### Dump the classifier model in Pickle file

In [16]:
import pickle
# save the classifier
with open('CIFAR10.pkl', 'wb') as fid:
    pickle.dump(model, fid)

#### Read the dump classifier

In [18]:
# in your prediction file                                                                                                                                                                                                           

with open('CIFAR10.pkl', 'rb') as f:
    clf = pickle.load(f)